In [2]:
%%capture
!pip install torch-geometric

In [3]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
import json
from tqdm import tqdm

In [70]:
file_paths = {
    "train_file_path": {
        "data_file_path": f"arc-agi_training_challenges.json",
        "target_file_path": f"arc-agi_training_solutions.json"
    },
    "val_file_path": {
        "data_file_path": f"arc-agi_evaluation_challenges.json",
        "target_file_path": f"arc-agi_evaluation_solutions.json"
    },
    "test_file_path": {
        "data_file_path": f"arc-agi_test_challenges.json"
    },
}

NameError: name 'BASE_FOLDER' is not defined

In [69]:
print("ARC Dataset Class")
class ARCDataset:

    def __init__(self, train_file_path, val_file_path, test_file_path, batch_size):
        self.output = {
            "train_output":{},
            "val_output":{}
        }
        self.origin_data = {}
        self.train_data = self.extract_file(train_file_path, "train")
        self.val_data = self.extract_file(val_file_path, "val")
        self.test_data = self.extract_file(test_file_path, "test")
        self.batch_size = batch_size

    #   for dataset class, we just need the input and output data
    def extract_data(self, data):
        d = []
        for key, inps, targ, index in data:
            d.append([inps, targ])
        return d

    def train_dataset(self):
        return DataLoader(self.extract_data(self.train_data), batch_size=self.batch_size, shuffle=True)

    def val_dataset(self):
        return DataLoader(self.extract_data(self.val_data), batch_size=self.batch_size, shuffle=False)

    def test_dataset(self):
        return self.test_data

    #   extract json file
    def extract_file(self, file_path, type_data):
        data_file_path = file_path["data_file_path"]
        target_file_path = file_path["target_file_path"] if type_data != "test" else None
        if target_file_path != None:
            with open(target_file_path, 'r') as f:
                sol = json.load(f)
            for i in sol.keys():
                self.output[f"{type_data}_output"][i] = sol[i]
        return self.load_data(data_file_path, type_data)

    def load_data(self, file_path, type_data):
        with open(file_path, 'r') as f:
            data = json.load(f)
        self.origin_data[type_data] = data
        return self.parse_data(data, type_data)

    #   add '0' value for padding. each row must have 30 length
    def expand_data(self, data, data_append=0):
        return np.array([*data, *[data_append for _ in range(30 - len(data))]])

    #   add '0' or np.zeros(30) so the data shape become (30,30) (900 after flatten)
    def prep_data(self, data):
        data = np.array(data)

        ndata = []
        for d in data:
            ndata.append(self.expand_data(d, 0))
        return torch.tensor(self.expand_data(ndata, np.zeros(30)).flatten())

    # the input data idea is give the nn example_input + example_target + test_input so LSTM can remember what it should do
    def parse_data(self, data, type_data):
        ndata = []
        for key in tqdm(data.keys(), desc=type_data):
            train_data = data[key]['train']
            test_data = data[key]['test']
            train_temp, test_temp = [], []
            for trd in train_data:
                input_tensor = self.prep_data(trd['input'])
                output_tensor = self.prep_data(trd['output'])
                train_temp.append([
                    input_tensor,
                    output_tensor
                ])
            for i in range(len(test_data)):
                input_tensor = self.prep_data(test_data[i]['input'])
                if type_data != 'test' and key in self.output[f"{type_data}_output"]:
                    output_tensor = self.prep_data(self.output[f"{type_data}_output"][key][i])
                else:
                    output_tensor = np.zeros(900)
                test_temp.append([
                    input_tensor,
                    output_tensor
                ])
            for i, trd_1 in enumerate(train_temp):
                for j, tsd in enumerate(test_temp):
                    ndata.append([key, torch.tensor([*[*trd_1[0], 10, *trd_1[1]], 11, *tsd[0], 10]), torch.tensor(tsd[1]), j])

        print(f"Data type: {type_data}. Unique Puzzle: {len(data.keys())}. Parsing Puzzle: {len(ndata)}")
        return ndata

ARC Dataset Class


In [68]:
dataset = ARCDataset(**file_paths)

TypeError: expected str, bytes or os.PathLike object, not dict